In [1]:
import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.layers import (
    Input, Conv2D, MaxPooling2D, Dropout, Flatten, Dense, BatchNormalization,
    GlobalAveragePooling2D, Reshape, multiply, Add, Activation,concatenate,ReLU,
    DepthwiseConv2D, SeparableConv2D  # Import DepthwiseConv2D here
)
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.utils import to_categorical
from tensorflow.keras import backend as K
from tensorflow.keras.callbacks import TensorBoard
import datetime
import tensorflow as tf
from tensorflow.keras.preprocessing.image import load_img, img_to_array
import numpy as np
from tensorflow.keras.optimizers import SGD 
from tensorflow.keras.models import load_model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau, LearningRateScheduler
import numpy as np
from tensorflow.keras import layers, models, regularizers
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.layers import Input, GlobalAveragePooling2D, Dense, Dropout
import subprocess
import time
import threading

In [2]:
import tkinter as tk
from tkinter import filedialog, messagebox
from PIL import Image, ImageTk
import numpy as np
import tensorflow as tf
from tensorflow.keras.preprocessing import image
import cv2
model_path = 'resnetdemo13.h5'
model = tf.keras.models.load_model(model_path,compile=False)
model.compile(optimizer=Adam(learning_rate=1e-5),  
              loss='categorical_crossentropy',
              metrics=['accuracy'])
labels={0: 'AUDI', 1: 'BAOJUN', 2: 'BMW', 3: 'BYD', 4: 'Benz', 5: 'Buick', 6: 'CHERY', 7: 'Cadillac', 8: 'Changan', 9: 'Chevrolet', 10: 'Citroen', 11: 'Coupe', 12: 'FAWHaima', 13: 'Ford', 14: 'GMC', 15: 'Geely', 16: 'HAVAL', 17: 'HONDA', 18: 'Hyundai', 19: 'INFINITI', 20: 'JEEP', 21: 'KIA', 22: 'Lexus', 23: 'Lincoln', 24: 'MG', 25: 'Mazda', 26: 'Mitsubishi', 27: 'Nissan', 28: 'PEUGEOT', 29: 'Porsche', 30: 'Qoros', 31: 'RANGEROVER', 32: 'ROEWE', 33: 'Renault', 34: 'SKODA', 35: 'SUZUKI', 36: 'Saab', 37: 'SoueastMotor', 38: 'Subaru', 39: 'TOYOTA', 40: 'TRUMCHI', 41: 'VOLVO', 42: 'Venucia', 43: 'Volkswagen', 44: 'WuLing'}
img_path = None 
def predict_vehicle_logo(image_path, model):
    global img_path
    if not image_path:
        messagebox.showerror("Error", "No image loaded.")
        return "No image loaded"
    try:
        img = image.load_img(image_path, target_size=(256, 256))
        img_array = image.img_to_array(img)
        img_array = np.expand_dims(img_array, axis=0)
        img_array /= 255.0
        predictions = model.predict(img_array)
        class_index = np.argmax(predictions, axis=1)[0]
        return labels[class_index]
    except Exception as e:
        messagebox.showerror("Error", f"Failed to predict logo: {e}")
        return f"Error: {e}"


def make_gradcam_heatmap(img_array, model, last_conv_layer_name, pred_index=None):
    grad_model = tf.keras.models.Model([model.inputs], [model.get_layer(last_conv_layer_name).output, model.output])
    with tf.GradientTape() as tape:
        last_conv_layer_output, preds = grad_model(img_array)
        if pred_index is None:
            pred_index = tf.argmax(preds[0])
        class_channel = preds[:, pred_index]
    grads = tape.gradient(class_channel, last_conv_layer_output)
    pooled_grads = tf.reduce_mean(grads, axis=(0, 1, 2))
    last_conv_layer_output = last_conv_layer_output[0]
    heatmap = last_conv_layer_output @ pooled_grads[..., tf.newaxis]
    heatmap = tf.squeeze(heatmap)
    heatmap = tf.maximum(heatmap, 0) / tf.math.reduce_max(heatmap)
    return heatmap.numpy()

def display_gradcam(img_path, heatmap, alpha=0.4):
    img = cv2.imread(img_path)
    img = cv2.resize(img, (256, 256))
    heatmap = np.uint8(255 * heatmap)
    jet = cv2.applyColorMap(heatmap, cv2.COLORMAP_JET)
    jet = cv2.resize(jet, (img.shape[1], img.shape[0]))
    superimposed_img = jet * alpha + img
    superimposed_img = np.clip(superimposed_img, 0, 255).astype(np.uint8)
    superimposed_img = cv2.cvtColor(superimposed_img, cv2.COLOR_BGR2RGB)
    return superimposed_img

def show_gradcam():
    img_array = image.img_to_array(Image.open(img_path).resize((256, 256)))
    img_array = np.expand_dims(img_array, axis=0) / 255.0
    heatmap = make_gradcam_heatmap(img_array, model, 'conv2d_45')
    gradcam_img = display_gradcam(img_path, heatmap)
    gradcam_img = Image.fromarray(gradcam_img)
    gradcam_img = ImageTk.PhotoImage(gradcam_img.resize((250, 250), Image.Resampling.LANCZOS))
    panel.configure(image=gradcam_img)
    panel.image = gradcam_img

def upload_image():
    global img_path, panel
    img_path = filedialog.askopenfilename()  # 弹出文件选择框让用户选择图片
    if img_path:
        img = Image.open(img_path)  # 使用PIL加载图片
        img = img.resize((256, 256), Image.Resampling.LANCZOS)
        img = ImageTk.PhotoImage(img)
        panel.configure(image=img)
        panel.image = img
    else:
        messagebox.showinfo("Upload Cancelled", "You have cancelled the upload of the image")
        img = Image.open('Not_uploaded.png')  # 使用PIL加载占位图片
        img = img.resize((256, 256), Image.LANCZOS)
        img = ImageTk.PhotoImage(img)
        original_panel.configure(image=img)
        original_panel.image = img
        result_label.config(text="Please upload pictures")
        return
        
def recognize_logo():
    global img_path
    result = predict_vehicle_logo(img_path, model)
    result_label.config(text=f"Prediction result: {result}")

    
root = tk.Tk()
root.title("Vehicle Logo Classification System")
root.geometry('400x400')

placeholder = Image.open('Not_uploaded.png')  # 加载一个占位符图片
placeholder = placeholder.resize((250, 250), Image.Resampling.LANCZOS)
placeholder = ImageTk.PhotoImage(placeholder)

upload_btn = tk.Button(root, text="Upload Image", command=upload_image)
upload_btn.pack()
recognize_btn = tk.Button(root, text="Recognize Logo", command=recognize_logo)
recognize_btn.pack()
recognize_btn = tk.Button(root, text="Show GradCAM", command=show_gradcam)
recognize_btn.pack()
result_label = tk.Label(root, text="The recognition result will be displayed here")
result_label.pack()
panel = tk.Label(root, image=placeholder)  # 使用占位符图片初始化标签
panel.pack()
root.mainloop()

1/1 [==============================] - 2s 2s/step
